In [ ]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
import tensorflow_decision_forests as tfdf

print(f"Found TF-DF {tfdf.__version__}")


train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
serving_df = pd.read_csv("/kaggle/input/titanic/test.csv")

train_df.head(10)



看看数据长什么样，
然后有3个函数

1. 一个规则化name
2. python的数组-1 index 是指到最后一个元素


In [ ]:
def preprocess(df):
    df = df.copy()
    
    def normalize_name(x):
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])
    
    def ticket_number(x):
        return x.split(" ")[-1]
        
    def ticket_item(x):
        items = x.split(" ")
        if len(items) == 1:
            return "NONE"
        return "_".join(items[0:-1])
    
    df["Name"] = df["Name"].apply(normalize_name)
    df["Ticket_number"] = df["Ticket"].apply(ticket_number)
    df["Ticket_item"] = df["Ticket"].apply(ticket_item)                     
    return df
    
preprocessed_train_df = preprocess(train_df)
preprocessed_serving_df = preprocess(serving_df)

preprocessed_train_df.head(5)

去掉一些对于模型的想法的不必要内容
买什么票和分给他的id和生还，与否先不管。
是train的数据集

In [ ]:
input_features = list(preprocessed_train_df.columns)
input_features.remove("Ticket")
input_features.remove("PassengerId")
input_features.remove("Survived")
#input_features.remove("Ticket_number")

print(f"Input features: {input_features}")

那么现在是主要探究，（一些名称不知道是什么😓）

pd呢，做了一些普通的数学，现在把数据放入tf试试

In [ ]:
# 现在我这个就看得懂，tokenize嘛，以前怎么看都不懂
def tokenize_names(features, labels=None):
    """Divite the names into tokens. TF-DF can consume text tokens natively."""
    features["Name"] =  tf.strings.split(features["Name"])
    return features, labels

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_df,label="Survived").map(tokenize_names)
serving_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_serving_df).map(tokenize_names)

上面这个开始魔了，features我们类比为x， label是标签作结果为y。

然后这里是一个特别的地方是 tf 自己的strings split，我称为他作信息量化，把数据中名字token化。

后面两句，为什么pd的dataframe不能和tf的dataset一样名字的概念啊

train_ds 用于模型训练，而 serving_ds 可能用于模型的推理或评估

下面， 我服了，无端端搞个梯度增压（瞎搞名字）模型，我理解为一群羊来到了一个树林，全部赶进去，羊崽们自己爱怎么走就怎么走，可能跟地形，风景啊相关，不过我们看看最终结果就可以了。

In [ ]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0, # Very few logs
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True, # Only use the features in "features"
    random_seed=1234,
)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")


下面的是加了很多参数去试试

In [ ]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0, # Very few logs
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True, # Only use the features in "features"
    
    #num_trees=2000,
    
    # Only for GBT.
    # A bit slower, but great to understand the model.
    # compute_permutation_variable_importance=True,
    
    # Change the default hyper-parameters
    # hyperparameter_template="benchmark_rank1@v1",
    
    #num_trees=1000,
    #tuner=tuner
    
    min_examples=1,
    categorical_algorithm="RANDOM",
    #max_depth=4,
    shrinkage=0.05,
    #num_candidate_attributes_ratio=0.2,
    split_axis="SPARSE_OBLIQUE",
    sparse_oblique_normalization="MIN_MAX",
    sparse_oblique_num_projections_exponent=2.0,
    num_trees=2000,
    #validation_ratio=0.0,
    random_seed=1234,
    
)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

下面看看情况，稍微看一点点对于这些参数的大概讲解。

> 先来一个个人思考。随机的策略下，收紧的5% ……

上面这个人的思考，完全不搭边。

我再大概说一次。选材features，把表单的里面都选进来，最低限度在一个样本及以后都去烹饪，在慢慢细嚼慢熬的每个以5%的占用影响率去反馈结果，所以需要熬的时间长一些，冒泡反馈的次数多一些。

对于其他的参数，"SPARSE_OBLIQUE"，稀稀拉拉的偏斜。
多个features来决定，我有一个想象的场景是，每个不同的feature是一个不同方向的向量，长度也不一样，而多个feature结合出来的这一个矩阵或者说几何形状吧，他们这个时候就是一个结晶点，是计算过程中的一个控制器。

那我设计一个小鸡走移动迷宫，这个移动迷宫是有意识的，不过它自己也不知道迷宫的出口在哪里，而且是很多很多的小鸡走出出口的口口相传之后，迷宫才会发生变化。所以，当遇到这个"sparse_oblique"的时候，当小鸡的身型和颜色等等的各种特性，它不是每次都全部把这种特性算进去计算，而是都留一些在下一个分岔口去选择决定。

In [ ]:
model.summary()